In [1]:
import folium
import pandas as pd

from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [2]:
#The following dataset with the name spacex_launch_geo.csv is an augmented dataset 
#with latitude and longitude added for each site.
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
df = pd.read_csv(url)
df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


## Task 1: Mark all launch sites on a map


In [3]:
#take a look at what are the coordinates for each site.
df = df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


In [4]:
#Let's visualize those locations by pinning them on a map
#create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [5]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [6]:
#let's add a circle and marker for each launch site in data frame launch_sites
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

for i, site in enumerate(launch_sites_df['Launch Site']):
    coord = [launch_sites_df.iloc[i,1], launch_sites_df.iloc[i,2]]
    circle = folium.Circle(coord, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(site))
    marker = folium.map.Marker(
        coord,
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site,
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map
#You can see that all sites are on the coats.

## Task 2: Mark the success/failed launches for each site on the map


In [7]:
#Note that a launch only happens in one of the four launch sites, 
#which means many launch records will have the exact same coordinate.
#Marker clusters are a good way to simplify a map containing many markers with the same coordinate.
#Let's first create a MarkerCluster object
marker_cluster = MarkerCluster()

In [8]:
#Create a new column in the df called marker_color to store the marker colors based on the class
#If a launch was successful (class=1), then we use a green marker 
#and if a launch failed, we use a red marker (class=0)

#I first create an empty list and populate it
marker_color = []
for value in df['class']:
    if value == 0:
        marker_color.append('red')
    else:
        marker_color.append('green')

#then I add this list as a column to the df
df['marker_color'] = marker_color

In [9]:
#Now, for each launch result in the df, I add a folium.Marker to marker_cluster
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in df I create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
for i, row in df.iterrows():
    coord = [df.iloc[i,1], df.iloc[i,2]]
    marker = folium.Marker(coord, icon=folium.Icon(color='white', icon_color=row['marker_color']))
    marker_cluster.add_child(marker)

site_map
#it doeasnt work

## TASK 3: Calculate the distances between a launch site to its proximities

In [10]:
#Let's first add a MousePosition on the map to get coordinate for a mouse over a point on the map

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [66]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [68]:
# find coordinate of the closet coastline
# e.g.,: 
Lat = 28.56367  
Lon = -80.57163
#not finished